In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import matplotlib
matplotlib.use('TkAgg')  # should fix matplotlib issues
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
import os
os.environ["WANDB_NOTEBOOK_NAME"] = "Catboost_Multi.ipynb"


pd.options.display.max_columns = 100

In [2]:
A_X_train_observed = pd.read_parquet('./data/raw/A/X_train_observed.parquet')
A_X_train_estimated = pd.read_parquet('./data/raw/A/X_train_estimated.parquet')
A_X_test_estimated = pd.read_parquet('./data/raw/A/X_test_estimated.parquet')
A_train_targets = pd.read_parquet('./data/raw/A/train_targets.parquet')
B_X_train_observed = pd.read_parquet('./data/raw/B/X_train_observed.parquet')
B_X_train_estimated = pd.read_parquet('./data/raw/B/X_train_estimated.parquet')
B_X_test_estimated = pd.read_parquet('./data/raw/B/X_test_estimated.parquet')
B_train_targets = pd.read_parquet('./data/raw/B/train_targets.parquet')
C_X_train_observed = pd.read_parquet('./data/raw/C/X_train_observed.parquet')
C_X_train_estimated = pd.read_parquet('./data/raw/C/X_train_estimated.parquet')
C_X_test_estimated = pd.read_parquet('./data/raw/C/X_test_estimated.parquet')
C_train_targets = pd.read_parquet('./data/raw/C/train_targets.parquet')

#Load df to predict
y_predictons = pd.read_csv('data/raw/test.csv')


In [3]:
y_predictons['time'] = pd.to_datetime(y_predictons['time'])

In [4]:
# remove noisy features
def remove_night_light_discrepancies(df: pd.DataFrame):
    # remove all rows where pv_measure
    
    # step 1: Identify runs of equal, non-zero values
    df["group"] = (
        (df["pv_measurement"] != df["pv_measurement"].shift())
        | (df["pv_measurement"] == 0)
    ).cumsum()

    # step 2: Count occurances in each run
    counts = df.groupby("group")["pv_measurement"].transform("count")

    # step 3: Identify groups to remove
    to_remove = (counts >= 6) & (df["pv_measurement"] != 0)

    # step 4: Remove rows
    df_cleaned = df[~to_remove].drop(columns=["group"])
    return df_cleaned

# set all sun_elevation that are less than 0 to 0
def set_sun_elevation_to_zero(df: pd.DataFrame):
    df.loc[df['sun_elevation:d'] < 0, 'sun_elevation:d'] = 0
    return df

def add_sun_elevation_is_day_feature(df: pd.DataFrame):
    df['sun_elevation_is_day'] = df['sun_elevation:d'] * (df['is_day:idx'] - df["is_in_shadow:idx"])
    df = df.drop(columns=['is_in_shadow:idx', "is_day:idx", "sun_elevation:d"])
    return df

def resample_and_pivot(df, time_col='date_forecast'):
    # Ensure the date_forecast is a datetime type
    df[time_col] = pd.to_datetime(df[time_col])
    
    # Create a column that represents the 15-minute segment of the hour (00, 15, 30, 45)
    df['minutes'] = df[time_col].dt.minute
    
    # Remove the original 'date_forecast' column to avoid conflicts
    df_without_date = df.drop(columns=[time_col])
    
    # Pivot the 15-minute data to wide format
    df_pivoted = df_without_date.pivot_table(index=[df[time_col].dt.floor('H')],
                                             columns='minutes',
                                             values=[col for col in df.columns if col not in [time_col, 'minutes']],
                                             aggfunc='first')
    
    # Flatten the MultiIndex columns
    df_pivoted.columns = ['_'.join(map(str, col)).strip() for col in df_pivoted.columns.values]
    
    # Reset index to turn the grouped hourly timestamps back into a column
    df_pivoted.reset_index(inplace=True)
    
    # Rename the 'date_forecast' column to avoid the ValueError
    df_pivoted.rename(columns={'index': time_col}, inplace=True)
    
    return df_pivoted

def resample_and_max(pivoted_df, features):
    for feature in features:
        # Select columns for the current feature of interest
        feature_cols = [col for col in pivoted_df.columns if feature in col]
        
        # Find the maximum value across the selected columns for each hour
        max_feature_col_name = feature + '_max'
        pivoted_df[max_feature_col_name] = pivoted_df[feature_cols].max(axis=1)
        
        # Drop the original 15-minute interval columns for the current feature
        pivoted_df.drop(columns=feature_cols, inplace=True)
    
    # The resulting dataframe will have all other features intact, and additional columns
    # with the maximum value for the specified features over each hour
    return pivoted_df

In [5]:
columns_to_keep_A = [
    # Directly related to Solar Energy
    "clear_sky_energy_1h:J",
    "clear_sky_rad:W",
    "diffuse_rad:W",
    "direct_rad:W", # max for hour
    "is_day:idx",
    "sun_azimuth:d",
    "sun_elevation:d",

    # Weather Conditions affecting Solar Panels
    "absolute_humidity_2m:gm3",
    "cloud_base_agl:m",
    "effective_cloud_cover:p",
    "is_in_shadow:idx",
    #"total_cloud_cover:p",

    # Temperature and Snow
    "dew_point_2m:K",
    #"fresh_snow_12h:cm",
    #"fresh_snow_1h:cm",
    #"fresh_snow_24h:cm",
    #"fresh_snow_3h:cm",
    #"fresh_snow_6h:cm",
    #"snow_depth:cm",
    "t_1000hPa:K",

    # Wind
    "wind_speed_10m:ms",

    # Miscellaneous
    "sfc_pressure:hPa",
    "date_forecast",
    "visibility:m",
]
columns_to_drop_A = []
for column in A_X_train_observed.columns:
    if column not in columns_to_keep_A:
        columns_to_drop_A.append(column)
print(columns_to_drop_A)

# Update DataFrames for solar panel A:
A_X_train_observed = A_X_train_observed.drop(columns=columns_to_drop_A)
A_X_train_estimated = A_X_train_estimated.drop(columns=columns_to_drop_A+['date_calc'])
A_X_test_estimated = A_X_test_estimated.drop(columns=columns_to_drop_A+['date_calc'])

A_train_targets = remove_night_light_discrepancies(A_train_targets)

# set sun_elevation to zero when the sun is below the horizon
A_X_train_observed = set_sun_elevation_to_zero(A_X_train_observed)
A_X_train_estimated = set_sun_elevation_to_zero(A_X_train_estimated)
A_X_test_estimated = set_sun_elevation_to_zero(A_X_test_estimated)

# add sun_elevation_is_day feature
A_X_train_observed = add_sun_elevation_is_day_feature(A_X_train_observed)
A_X_train_estimated = add_sun_elevation_is_day_feature(A_X_train_estimated)
A_X_test_estimated = add_sun_elevation_is_day_feature(A_X_test_estimated)

# Apply the function to each DataFrame
A_X_train_observed = resample_and_pivot(A_X_train_observed)
A_X_train_estimated = resample_and_pivot(A_X_train_estimated)
A_X_test_estimated = resample_and_pivot(A_X_test_estimated)

# Apply the function to each DataFrame
A_X_train_observed = resample_and_max(A_X_train_observed, features=['direct_rad:W'])
A_X_train_estimated = resample_and_max(A_X_train_estimated, features=['direct_rad:W'])
A_X_test_estimated = resample_and_max(A_X_test_estimated, features=['direct_rad:W'])

['air_density_2m:kgm3', 'ceiling_height_agl:m', 'dew_or_rime:idx', 'diffuse_rad_1h:J', 'direct_rad_1h:J', 'elevation:m', 'fresh_snow_12h:cm', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm', 'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'msl_pressure:hPa', 'precip_5min:mm', 'precip_type_5min:idx', 'pressure_100m:hPa', 'pressure_50m:hPa', 'prob_rime:p', 'rain_water:kgm2', 'relative_humidity_1000hPa:p', 'snow_density:kgm3', 'snow_depth:cm', 'snow_drift:idx', 'snow_melt_10min:mm', 'snow_water:kgm2', 'super_cooled_liquid_water:kgm2', 'total_cloud_cover:p', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms', 'wind_speed_w_1000hPa:ms']


In [6]:
columns_to_keep_B = [
    # Directly related to Solar Energy
    "clear_sky_energy_1h:J",
    "clear_sky_rad:W",
    "diffuse_rad:W",
    "direct_rad:W", # max for hour
    "is_day:idx",
    "sun_azimuth:d",
    "sun_elevation:d",

    # Weather Conditions affecting Solar Panels
    "absolute_humidity_2m:gm3",
    "cloud_base_agl:m",
    "effective_cloud_cover:p",
    "is_in_shadow:idx",
    #"total_cloud_cover:p",

    # Temperature and Snow
    "dew_point_2m:K",
    #"fresh_snow_12h:cm",
    #"fresh_snow_1h:cm",
    #"fresh_snow_24h:cm",
    #"fresh_snow_3h:cm",
    #"fresh_snow_6h:cm",
    #"snow_depth:cm",
    "t_1000hPa:K",

    # Wind
    "wind_speed_10m:ms",

    # Miscellaneous
    "sfc_pressure:hPa",
    "date_forecast",
    "visibility:m",
]
columns_to_drop_B = []
for column in B_X_train_observed.columns:
    if column not in columns_to_keep_B:
        columns_to_drop_B.append(column)
print(columns_to_drop_B)

# Update DataFrames for solar panel A:
B_X_train_observed = B_X_train_observed.drop(columns=columns_to_drop_B)
B_X_train_estimated = B_X_train_estimated.drop(columns=columns_to_drop_B+['date_calc'])
B_X_test_estimated = B_X_test_estimated.drop(columns=columns_to_drop_B+['date_calc'])

B_train_targets = remove_night_light_discrepancies(B_train_targets)

# set sun_elevation to zero when the sun is below the horizon
B_X_train_observed = set_sun_elevation_to_zero(B_X_train_observed)
B_X_train_estimated = set_sun_elevation_to_zero(B_X_train_estimated)
B_X_test_estimated = set_sun_elevation_to_zero(B_X_test_estimated)

# add sun_elevation_is_day feature
B_X_train_observed = add_sun_elevation_is_day_feature(B_X_train_observed)
B_X_train_estimated = add_sun_elevation_is_day_feature(B_X_train_estimated)
B_X_test_estimated = add_sun_elevation_is_day_feature(B_X_test_estimated)

# Apply the function to each DataFrame
B_X_train_observed = resample_and_pivot(B_X_train_observed)
B_X_train_estimated = resample_and_pivot(B_X_train_estimated)
B_X_test_estimated = resample_and_pivot(B_X_test_estimated)

# Apply the function to each DataFrame
B_X_train_observed = resample_and_max(B_X_train_observed, features=['direct_rad:W'])
B_X_train_estimated = resample_and_max(B_X_train_estimated, features=['direct_rad:W'])
B_X_test_estimated = resample_and_max(B_X_test_estimated, features=['direct_rad:W'])

['air_density_2m:kgm3', 'ceiling_height_agl:m', 'dew_or_rime:idx', 'diffuse_rad_1h:J', 'direct_rad_1h:J', 'elevation:m', 'fresh_snow_12h:cm', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm', 'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'msl_pressure:hPa', 'precip_5min:mm', 'precip_type_5min:idx', 'pressure_100m:hPa', 'pressure_50m:hPa', 'prob_rime:p', 'rain_water:kgm2', 'relative_humidity_1000hPa:p', 'snow_density:kgm3', 'snow_depth:cm', 'snow_drift:idx', 'snow_melt_10min:mm', 'snow_water:kgm2', 'super_cooled_liquid_water:kgm2', 'total_cloud_cover:p', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms', 'wind_speed_w_1000hPa:ms']


In [7]:
columns_to_keep_C = [
    # Directly related to Solar Energy
    "clear_sky_energy_1h:J",
    "clear_sky_rad:W",
    "diffuse_rad:W",
    "direct_rad:W", # max for hour
    "is_day:idx",
    "sun_azimuth:d",
    "sun_elevation:d",

    # Weather Conditions affecting Solar Panels
    "absolute_humidity_2m:gm3",
    "cloud_base_agl:m",
    "effective_cloud_cover:p",
    "is_in_shadow:idx",
    #"total_cloud_cover:p",

    # Temperature and Snow
    "dew_point_2m:K",
    #"fresh_snow_12h:cm",
    #"fresh_snow_1h:cm",
    #"fresh_snow_24h:cm",
    #"fresh_snow_3h:cm",
    #"fresh_snow_6h:cm",
    #"snow_depth:cm",
    "t_1000hPa:K",

    # Wind
    "wind_speed_10m:ms",

    # Miscellaneous
    "sfc_pressure:hPa",
    "date_forecast",
    "visibility:m",
]
columns_to_drop_C = []
for column in C_X_train_observed.columns:
    if column not in columns_to_keep_C:
        columns_to_drop_C.append(column)
print(columns_to_drop_C)

# Update DataFrames for solar panel A:
C_X_train_observed = C_X_train_observed.drop(columns=columns_to_drop_C)
C_X_train_estimated = C_X_train_estimated.drop(columns=columns_to_drop_C+['date_calc'])
C_X_test_estimated = C_X_test_estimated.drop(columns=columns_to_drop_C+['date_calc'])

C_train_targets = remove_night_light_discrepancies(C_train_targets)

# set sun_elevation to zero when the sun is below the horizon
C_X_train_observed = set_sun_elevation_to_zero(C_X_train_observed)
C_X_train_estimated = set_sun_elevation_to_zero(C_X_train_estimated)
C_X_test_estimated = set_sun_elevation_to_zero(C_X_test_estimated)

# add sun_elevation_is_day feature
C_X_train_observed = add_sun_elevation_is_day_feature(C_X_train_observed)
C_X_train_estimated = add_sun_elevation_is_day_feature(C_X_train_estimated)
C_X_test_estimated = add_sun_elevation_is_day_feature(C_X_test_estimated)

# Apply the function to each DataFrame
C_X_train_observed = resample_and_pivot(C_X_train_observed)
C_X_train_estimated = resample_and_pivot(C_X_train_estimated)
C_X_test_estimated = resample_and_pivot(C_X_test_estimated)

# Apply the function to each DataFrame
C_X_train_observed = resample_and_max(C_X_train_observed, features=['direct_rad:W'])
C_X_train_estimated = resample_and_max(C_X_train_estimated, features=['direct_rad:W'])
C_X_test_estimated = resample_and_max(C_X_test_estimated, features=['direct_rad:W'])

['air_density_2m:kgm3', 'ceiling_height_agl:m', 'dew_or_rime:idx', 'diffuse_rad_1h:J', 'direct_rad_1h:J', 'elevation:m', 'fresh_snow_12h:cm', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm', 'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'msl_pressure:hPa', 'precip_5min:mm', 'precip_type_5min:idx', 'pressure_100m:hPa', 'pressure_50m:hPa', 'prob_rime:p', 'rain_water:kgm2', 'relative_humidity_1000hPa:p', 'snow_density:kgm3', 'snow_depth:cm', 'snow_drift:idx', 'snow_melt_10min:mm', 'snow_water:kgm2', 'super_cooled_liquid_water:kgm2', 'total_cloud_cover:p', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms', 'wind_speed_w_1000hPa:ms']


In [8]:
# Encode the categorical variable 'solar_pannel_location'
A_X_train_observed['solar_pannel_location'] = 'A'
A_X_train_estimated['solar_pannel_location'] = 'A'
A_X_test_estimated['solar_pannel_location'] = 'A'
B_X_train_observed['solar_pannel_location'] = 'B'
B_X_train_estimated['solar_pannel_location'] = 'B'
B_X_test_estimated['solar_pannel_location'] = 'B'
C_X_train_observed['solar_pannel_location'] = 'C'
C_X_train_estimated['solar_pannel_location'] = 'C'
C_X_test_estimated['solar_pannel_location'] = 'C'

# Encode the categorical variable 'solar_pannel_location'
A_train_targets['solar_pannel_location'] = 'A'
B_train_targets['solar_pannel_location'] = 'B'
C_train_targets['solar_pannel_location'] = 'C'


In [9]:
# Encode the categorical variable 'is_estimated'
A_X_train_observed['is_estimated'] = False
A_X_train_estimated['is_estimated'] = True
B_X_train_observed['is_estimated'] = False
B_X_train_estimated['is_estimated'] = True
C_X_train_observed['is_estimated'] = False
C_X_train_estimated['is_estimated'] = True

In [10]:
# A_X_train df's combined:
A_X_TRAIN = pd.concat([
    A_X_train_observed,
    A_X_train_estimated
], axis=0, ignore_index=True)

# B_X_train df's combined:
B_X_TRAIN = pd.concat([
    B_X_train_observed,
    B_X_train_estimated
], axis=0, ignore_index=True)

# C_X_train df's combined:
C_X_TRAIN = pd.concat([
    C_X_train_observed,
    C_X_train_estimated
], axis=0, ignore_index=True)


In [11]:
# Add time from apex
# Convert 'date_forecast' column to datetime
A_X_TRAIN['date_forecast'] = pd.to_datetime(A_X_TRAIN['date_forecast'])
B_X_TRAIN['date_forecast'] = pd.to_datetime(B_X_TRAIN['date_forecast'])
C_X_TRAIN['date_forecast'] = pd.to_datetime(C_X_TRAIN['date_forecast'])


# Convert 'time' column to datetime
A_X_test_estimated['date_forecast'] = pd.to_datetime(A_X_test_estimated['date_forecast'])
B_X_test_estimated['date_forecast'] = pd.to_datetime(B_X_test_estimated['date_forecast'])
C_X_test_estimated['date_forecast'] = pd.to_datetime(C_X_test_estimated['date_forecast'])

# Calculate the number of days away from day 171, considering year-end
A_X_TRAIN['days_from_apex'] = (A_X_TRAIN['date_forecast'].dt.dayofyear - 171).abs()
A_X_TRAIN['days_from_apex'] = A_X_TRAIN['days_from_apex'].apply(lambda x: min(x, 365 - x))  # Correct for year-end
B_X_TRAIN['days_from_apex'] = (B_X_TRAIN['date_forecast'].dt.dayofyear - 171).abs()
B_X_TRAIN['days_from_apex'] = B_X_TRAIN['days_from_apex'].apply(lambda x: min(x, 365 - x))  # Correct for year-end
C_X_TRAIN['days_from_apex'] = (C_X_TRAIN['date_forecast'].dt.dayofyear - 171).abs()
C_X_TRAIN['days_from_apex'] = C_X_TRAIN['days_from_apex'].apply(lambda x: min(x, 365 - x))  # Correct for year-end

A_X_test_estimated['days_from_apex'] = (A_X_test_estimated['date_forecast'].dt.dayofyear - 171).abs()
A_X_test_estimated['days_from_apex'] = A_X_test_estimated['days_from_apex'].apply(lambda x: min(x, 365 - x))  # Correct for year-end
B_X_test_estimated['days_from_apex'] = (B_X_test_estimated['date_forecast'].dt.dayofyear - 171).abs()
B_X_test_estimated['days_from_apex'] = B_X_test_estimated['days_from_apex'].apply(lambda x: min(x, 365 - x))  # Correct for year-end
C_X_test_estimated['days_from_apex'] = (C_X_test_estimated['date_forecast'].dt.dayofyear - 171).abs()
C_X_test_estimated['days_from_apex'] = C_X_test_estimated['days_from_apex'].apply(lambda x: min(x, 365 - x))  # Correct for year-end

# Define the apex time (11:00:00)
apex_time = pd.to_datetime('11:00:00').time()

# Calculate the time difference from the apex time
A_X_TRAIN['hours_from_apex'] = (abs(A_X_TRAIN['date_forecast'].dt.hour - apex_time.hour)) % 24
B_X_TRAIN['hours_from_apex'] = (abs(B_X_TRAIN['date_forecast'].dt.hour - apex_time.hour)) % 24
C_X_TRAIN['hours_from_apex'] = (abs(C_X_TRAIN['date_forecast'].dt.hour - apex_time.hour)) % 24

A_X_test_estimated['hours_from_apex'] = (abs(A_X_test_estimated['date_forecast'].dt.hour - apex_time.hour)) % 24
B_X_test_estimated['hours_from_apex'] = (abs(B_X_test_estimated['date_forecast'].dt.hour - apex_time.hour)) % 24
C_X_test_estimated['hours_from_apex'] = (abs(C_X_test_estimated['date_forecast'].dt.hour - apex_time.hour)) % 24

In [12]:
# Convert 'date_forecast' column to string
A_X_TRAIN['date_forecast'] = A_X_TRAIN['date_forecast'].astype(str)
B_X_TRAIN['date_forecast'] = B_X_TRAIN['date_forecast'].astype(str)
C_X_TRAIN['date_forecast'] = C_X_TRAIN['date_forecast'].astype(str)

# Convert 'date_forecast' column to string
A_X_test_estimated['date_forecast'] = A_X_test_estimated['date_forecast'].astype(str)
B_X_test_estimated['date_forecast'] = B_X_test_estimated['date_forecast'].astype(str)
C_X_test_estimated['date_forecast'] = C_X_test_estimated['date_forecast'].astype(str)

# Convert 'time' column to string
A_train_targets['time'] = A_train_targets['time'].astype(str)
B_train_targets['time'] = B_train_targets['time'].astype(str)
C_train_targets['time'] = C_train_targets['time'].astype(str)

A_X_TRAIN.columns

Index(['date_forecast', 'absolute_humidity_2m:gm3_0',
       'absolute_humidity_2m:gm3_15', 'absolute_humidity_2m:gm3_30',
       'absolute_humidity_2m:gm3_45', 'clear_sky_energy_1h:J_0',
       'clear_sky_energy_1h:J_15', 'clear_sky_energy_1h:J_30',
       'clear_sky_energy_1h:J_45', 'clear_sky_rad:W_0', 'clear_sky_rad:W_15',
       'clear_sky_rad:W_30', 'clear_sky_rad:W_45', 'cloud_base_agl:m_0',
       'cloud_base_agl:m_15', 'cloud_base_agl:m_30', 'cloud_base_agl:m_45',
       'dew_point_2m:K_0', 'dew_point_2m:K_15', 'dew_point_2m:K_30',
       'dew_point_2m:K_45', 'diffuse_rad:W_0', 'diffuse_rad:W_15',
       'diffuse_rad:W_30', 'diffuse_rad:W_45', 'effective_cloud_cover:p_0',
       'effective_cloud_cover:p_15', 'effective_cloud_cover:p_30',
       'effective_cloud_cover:p_45', 'sfc_pressure:hPa_0',
       'sfc_pressure:hPa_15', 'sfc_pressure:hPa_30', 'sfc_pressure:hPa_45',
       'sun_azimuth:d_0', 'sun_azimuth:d_15', 'sun_azimuth:d_30',
       'sun_azimuth:d_45', 'sun_elevation_

In [13]:

A_X_AND_Y_TRAIN = pd.merge(A_X_TRAIN, A_train_targets, left_on=['date_forecast', 'solar_pannel_location'], right_on=['time', 'solar_pannel_location'], how='inner')
B_X_AND_Y_TRAIN = pd.merge(B_X_TRAIN, B_train_targets, left_on=['date_forecast', 'solar_pannel_location'], right_on=['time', 'solar_pannel_location'], how='inner')
C_X_AND_Y_TRAIN = pd.merge(C_X_TRAIN, C_train_targets, left_on=['date_forecast', 'solar_pannel_location'], right_on=['time', 'solar_pannel_location'], how='inner')
A_X_AND_Y_TRAIN.columns

Index(['date_forecast', 'absolute_humidity_2m:gm3_0',
       'absolute_humidity_2m:gm3_15', 'absolute_humidity_2m:gm3_30',
       'absolute_humidity_2m:gm3_45', 'clear_sky_energy_1h:J_0',
       'clear_sky_energy_1h:J_15', 'clear_sky_energy_1h:J_30',
       'clear_sky_energy_1h:J_45', 'clear_sky_rad:W_0', 'clear_sky_rad:W_15',
       'clear_sky_rad:W_30', 'clear_sky_rad:W_45', 'cloud_base_agl:m_0',
       'cloud_base_agl:m_15', 'cloud_base_agl:m_30', 'cloud_base_agl:m_45',
       'dew_point_2m:K_0', 'dew_point_2m:K_15', 'dew_point_2m:K_30',
       'dew_point_2m:K_45', 'diffuse_rad:W_0', 'diffuse_rad:W_15',
       'diffuse_rad:W_30', 'diffuse_rad:W_45', 'effective_cloud_cover:p_0',
       'effective_cloud_cover:p_15', 'effective_cloud_cover:p_30',
       'effective_cloud_cover:p_45', 'sfc_pressure:hPa_0',
       'sfc_pressure:hPa_15', 'sfc_pressure:hPa_30', 'sfc_pressure:hPa_45',
       'sun_azimuth:d_0', 'sun_azimuth:d_15', 'sun_azimuth:d_30',
       'sun_azimuth:d_45', 'sun_elevation_

In [14]:
#Create a x_train_to_predict dataframe with the data to predict: 
A_X_test_estimated['date_forecast'] = A_X_test_estimated['date_forecast'].astype(str)
B_X_test_estimated['date_forecast'] = B_X_test_estimated['date_forecast'].astype(str)
C_X_test_estimated['date_forecast'] = C_X_test_estimated['date_forecast'].astype(str)
y_predictons['time'] = y_predictons['time'].astype(str)


A_X_test_estimated = pd.merge(A_X_test_estimated, y_predictons, left_on=['solar_pannel_location', 'date_forecast'], right_on=['location', 'time'], how='inner')
B_X_test_estimated = pd.merge(B_X_test_estimated, y_predictons, left_on=['solar_pannel_location', 'date_forecast'], right_on=['location', 'time'], how='inner')
C_X_test_estimated = pd.merge(C_X_test_estimated, y_predictons, left_on=['solar_pannel_location', 'date_forecast'], right_on=['location', 'time'], how='inner')


A_X_TEST_ENCODED = A_X_test_estimated.copy()
A_X_TEST_ENCODED.set_index('id', inplace=True)

B_X_TEST_ENCODED = B_X_test_estimated.copy()
B_X_TEST_ENCODED.set_index('id', inplace=True)

C_X_TEST_ENCODED = C_X_test_estimated.copy()
C_X_TEST_ENCODED.set_index('id', inplace=True)




B_X_TEST_ENCODED.head(5)


,date_forecast,absolute_humidity_2m:gm3_0,absolute_humidity_2m:gm3_15,absolute_humidity_2m:gm3_30,absolute_humidity_2m:gm3_45,clear_sky_energy_1h:J_0,clear_sky_energy_1h:J_15,clear_sky_energy_1h:J_30,clear_sky_energy_1h:J_45,clear_sky_rad:W_0,clear_sky_rad:W_15,clear_sky_rad:W_30,clear_sky_rad:W_45,cloud_base_agl:m_0,cloud_base_agl:m_15,cloud_base_agl:m_30,cloud_base_agl:m_45,dew_point_2m:K_0,dew_point_2m:K_15,dew_point_2m:K_30,dew_point_2m:K_45,diffuse_rad:W_0,diffuse_rad:W_15,diffuse_rad:W_30,diffuse_rad:W_45,effective_cloud_cover:p_0,effective_cloud_cover:p_15,effective_cloud_cover:p_30,effective_cloud_cover:p_45,sfc_pressure:hPa_0,sfc_pressure:hPa_15,sfc_pressure:hPa_30,sfc_pressure:hPa_45,sun_azimuth:d_0,sun_azimuth:d_15,sun_azimuth:d_30,sun_azimuth:d_45,sun_elevation_is_day_0,sun_elevation_is_day_15,sun_elevation_is_day_30,sun_elevation_is_day_45,t_1000hPa:K_0,t_1000hPa:K_15,t_1000hPa:K_30,t_1000hPa:K_45,visibility:m_0,visibility:m_15,visibility:m_30,visibility:m_45,wind_speed_10m:ms_0,wind_speed_10m:ms_15,wind_speed_10m:ms_30,wind_speed_10m:ms_45,direct_rad:W_max,solar_pannel_location,days_from_apex,hours_from_apex,time,prediction,location
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
720,2023-05-01 00:00:00,4.3,4.3,4.3,4.3,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1039.900024,1053.099976,1066.400024,1079.599976,271.700012,271.700012,271.600006,271.600006,0.0,0.000000,0.000000,0.000000,80.699997,77.099998,73.199997,69.000000,1013.200012,1013.0,1012.900024,1012.799988,10.521000,14.204000,17.870001,21.514000,-0.000,-0.000,-0.000,-0.000,273.799988,273.799988,273.799988,273.799988,31329.500000,31181.599609,31033.599609,30885.699219,4.0,4.0,3.9,3.9,0.0,B,50,11,2023-05-01 00:00:00,0,B
721,2023-05-01 01:00:00,4.3,4.3,4.2,4.2,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1092.900024,1080.099976,1067.300049,1054.500000,271.600006,271.500000,271.399994,271.299988,0.0,0.000000,0.000000,0.000000,64.599998,74.099998,82.300003,89.000000,1012.700012,1012.5,1012.400024,1012.299988,25.135000,28.730000,32.294998,35.831001,-0.000,-0.000,-0.000,-0.000,273.799988,273.799988,273.799988,273.799988,30737.800781,30519.000000,30300.300781,30081.699219,3.9,3.8,3.8,3.8,0.0,B,50,10,2023-05-01 01:00:00,0,B
722,2023-05-01 02:00:00,4.2,4.2,4.1,4.1,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1041.699951,1147.300049,1252.900024,1358.500000,271.200012,271.100006,271.000000,270.899994,0.0,0.000000,0.000000,0.000000,94.300003,90.699997,86.300003,81.099998,1012.200012,1012.0,1011.900024,1011.700012,39.335999,42.810001,46.255001,49.669998,-0.000,-0.000,-0.000,-0.000,273.799988,273.799988,273.899994,273.899994,29863.199219,30849.400391,31835.699219,32822.300781,3.7,3.7,3.6,3.6,0.0,B,50,9,2023-05-01 02:00:00,0,B
723,2023-05-01 03:00:00,4.1,4.0,4.0,4.0,0.000000,1331.099976,10260.400391,28918.699219,0.0,4.800000,14.900000,27.000000,1464.000000,1274.000000,1084.000000,894.000000,270.799988,270.700012,270.600006,270.500000,0.0,3.100000,12.900000,21.500000,75.099998,70.800003,66.599998,62.599998,1011.599976,1011.5,1011.299988,1011.200012,53.058998,56.422001,59.764000,63.088001,-0.000,0.000,2.101,3.577,273.899994,273.899994,273.899994,273.899994,33809.000000,34253.300781,34697.398438,35142.398438,3.6,3.5,3.5,3.4,5.8,B,50,8,2023-05-01 03:00:00,0,B
724,2023-05-01 04:00:00,3.9,3.9,3.9,3.9,59786.398438,106073.203125,164697.703125,236499.796875,43.0,63.299999,87.199997,114.099998,703.900024,847.400024,990.900024,1134.400024,270.299988,270.399994,270.399994,270.399994,31.4,42.099998,52.299999,63.799999,58.700001,63.799999,68.900002,74.000000,1011.099976,1011.0,1010.900024,1010.799988,66.397003,69.695999,72.989998,76.281998,5.098,6.657,8.250,9.871,273.899994,273.899994,273.899994,274.000000,35603.398438,35241.898438,34891.101562,34539.898438,3.4,3.3,3.3,3.3,42.5,B,50,7,2023-05-01 04:00:00,0,B


In [15]:

A_X_TEST_ENCODED = A_X_TEST_ENCODED.drop(['date_forecast','location', 'prediction'], axis=1)
B_X_TEST_ENCODED = B_X_TEST_ENCODED.drop(['date_forecast','location', 'prediction'], axis=1)
C_X_TEST_ENCODED = C_X_TEST_ENCODED.drop(['date_forecast','location', 'prediction'], axis=1)


#Drop all rows in training data with no tarhet value:
A_X_AND_Y_TRAIN = A_X_AND_Y_TRAIN.dropna(subset=['pv_measurement'])
B_X_AND_Y_TRAIN = B_X_AND_Y_TRAIN.dropna(subset=['pv_measurement'])
C_X_AND_Y_TRAIN = C_X_AND_Y_TRAIN.dropna(subset=['pv_measurement'])

# Load our

In [16]:
%pip install PyWavelets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.data.data_fetcher import get_raw_data
from src.features.preprocess_data import fetch_preprocessed_data, get_preprocessed_test_data

from src.visualization.plotting import plot_correlation_matrix


train_a, train_b, train_c, X_train_estimated_a, X_train_estimated_b, X_train_estimated_c, X_train_observed_a, X_train_observed_b, X_train_observed_c, X_test_estimated_a, X_test_estimated_b, X_test_estimated_c = get_raw_data()
X_train_obs_combined, X_val_obs_combined, y_train_obs_combined, y_val_obs_combined, X_train_est_combined, X_val_est_combined, y_train_est_combined, y_val_est_combined = fetch_preprocessed_data()
X_test_est_combined = get_preprocessed_test_data()


x_whole = pd.concat([X_train_obs_combined, X_val_obs_combined, X_train_est_combined, X_val_est_combined])
y_whole = pd.concat([y_train_obs_combined, y_val_obs_combined, y_train_est_combined, y_val_est_combined])
estimated_data = pd.concat([X_train_est_combined, X_val_est_combined, y_train_est_combined, y_val_est_combined])
# x_whole.reset_index(drop=True, inplace=True)
# y_whole.reset_index(drop=True, inplace=True)
pd.set_option('display.max_columns', None)


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
train_observed contains: time                                  0
pv_measurement                     6064
date_forecast                     10999
absolute_humidity_2m:gm3          10999
air_density_2m:kgm3               10999
clear_sky_energy_1h:J             10999
clear_sky_rad:W                   10999
cloud_base_agl:m                      0
dew_or_rime:idx                   10999
dew_point_2m:K                    10999
diffuse_rad:W                     10999
diffuse_rad_1h:J                  10999
direct_rad:W                      10999
direct_rad_1h:J                   10999
effective_cloud_cover:p           10999
fresh_snow_12h:cm                 10999
fresh_snow_1h:cm                  10999
fresh_snow_24h:cm                 10999
fresh_snow_3h:cm                  10999
fresh_snow_6h:cm                  10999
is_day:idx                        10999
is_in_shadow:idx                  10999
msl_pressure:hPa             

ValueError: NaTType does not support timetuple

# Differences in trainings data

In [ ]:
x_whole.describe()

,absolute_humidity_2m:gm3,air_density_2m:kgm3,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,is_day:idx,is_in_shadow:idx,precip_5min:mm,precip_type_5min:idx,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sun_azimuth:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,location_a,location_b,location_c,sin_day_of_year,cos_day_of_year,sin_hour,cos_hour,sun_product,modified_solar_elevation,effective_radiation,time_since_prediction,cloud_ratio,cloud_cover_over_30%,sun_addition,direct_rad_cloud_interaction,modified_solar_elevation_squared,is_freezing,is_snow,is_rain
count,77044.000000,77044.000000,7.704400e+04,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,7.704400e+04,77044.000000,7.704400e+04,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,7.704400e+04,77044.000000,77044.000000,77044.000000,77044.0,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000,77044.000000
mean,6.241166,1.252290,5.350376e+05,148.533371,1639.446899,0.007885,275.821411,39.974041,1.440989e+05,52.259331,1.881972e+05,66.943527,0.482267,0.565437,0.005829,0.084221,1001.689941,0.744772,0.010283,73.923294,179.718307,0.059241,279.985352,73.753952,32999.046875,3.012385,0.712736,0.666015,0.000034,0.384871,0.326034,0.289095,-0.011300,0.027707,-0.005012,1.618773e-02,6231.929688,0.155913,0.162052,0.0,0.863935,0.805176,92.233360,3015.051025,0.252298,0.124448,0.102967,0.099021
std,2.728918,0.036447,8.433045e+05,234.835953,1794.949219,0.237469,6.771035,61.647190,2.193565e+05,116.097191,4.125532e+05,34.011616,0.485604,0.483360,0.030001,0.322178,12.852131,5.278304,0.042884,14.194102,98.448479,0.107867,6.549464,33.702427,17984.406250,1.743064,2.751968,1.884144,0.006123,0.486568,0.468763,0.453345,0.708126,0.705461,0.709319,7.046936e-01,14842.071289,0.225102,0.261199,0.0,0.230534,0.396068,159.890182,8610.323242,0.303743,0.330095,0.303918,0.298693
min,0.500000,1.139250,0.000000e+00,0.000000,0.000000,-1.000000,247.425003,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,935.750000,0.000000,0.000000,19.575001,6.983000,0.000000,258.024994,0.000000,132.375000,0.025000,-7.225000,-8.400000,-0.100000,0.000000,0.000000,0.000000,-0.999999,-0.999979,-1.000000,-1.000000e+00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.200000,1.227500,0.000000e+00,0.000000,512.737503,0.000000,271.299988,0.000000,0.000000e+00,0.000000,0.000000e+00,42.150002,0.000000,0.000000,0.000000,0.000000,993.900024,0.000000,0.000000,64.599998,93.378752,0.000000,275.399994,53.700001,16443.712402,1.668750,-1.300000,-0.600000,0.000000,0.000000,0.000000,0.000000,-0.726225,-0.673275,-0.707107,-7.071068e-01,0.000000,0.000000,0.000000,0.0,0.809969,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.700000,1.251000,1.008428e+04,1.550000,1109.150024,0.000000,275.725006,0.850000,9.893438e+03,0.000000,0.000000e+00,79.362503,0.250000,1.000000,0.000000,0.000000,1002.700012,0.000000,0.000000,76.337502,180.095375,0.000000,279.375000,92.599998,37146.837891,2.675000,0.350000,0.700000,0.000000,0.000000,0.000000,0.000000,-0.004301,0.052658,0.000000,6.123234e-17,0.000000,0.000000,0.000000,0.0,0.991500,1.000000,0.875000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.075001,1.275250,8.512765e+05,234.399994,2000.993744,0.000000,281.000000,66.425003,2.389334e+05,31.556251,1.221875e+05,98.324997,1.000000,1.000000,0.000000,0.000000,1010.650024,0.000000,0.000000

In [ ]:
other_train = pd.concat([A_X_AND_Y_TRAIN, B_X_AND_Y_TRAIN, C_X_AND_Y_TRAIN])
other_train.describe()


,absolute_humidity_2m:gm3_0,absolute_humidity_2m:gm3_15,absolute_humidity_2m:gm3_30,absolute_humidity_2m:gm3_45,clear_sky_energy_1h:J_0,clear_sky_energy_1h:J_15,clear_sky_energy_1h:J_30,clear_sky_energy_1h:J_45,clear_sky_rad:W_0,clear_sky_rad:W_15,clear_sky_rad:W_30,clear_sky_rad:W_45,cloud_base_agl:m_0,cloud_base_agl:m_15,cloud_base_agl:m_30,cloud_base_agl:m_45,dew_point_2m:K_0,dew_point_2m:K_15,dew_point_2m:K_30,dew_point_2m:K_45,diffuse_rad:W_0,diffuse_rad:W_15,diffuse_rad:W_30,diffuse_rad:W_45,effective_cloud_cover:p_0,effective_cloud_cover:p_15,effective_cloud_cover:p_30,effective_cloud_cover:p_45,sfc_pressure:hPa_0,sfc_pressure:hPa_15,sfc_pressure:hPa_30,sfc_pressure:hPa_45,sun_azimuth:d_0,sun_azimuth:d_15,sun_azimuth:d_30,sun_azimuth:d_45,sun_elevation_is_day_0,sun_elevation_is_day_15,sun_elevation_is_day_30,sun_elevation_is_day_45,t_1000hPa:K_0,t_1000hPa:K_15,t_1000hPa:K_30,t_1000hPa:K_45,visibility:m_0,visibility:m_15,visibility:m_30,visibility:m_45,wind_speed_10m:ms_0,wind_speed_10m:ms_15,wind_speed_10m:ms_30,wind_speed_10m:ms_45,direct_rad:W_max,days_from_apex,hours_from_apex,pv_measurement
count,89736.000000,89736.000000,89736.000000,89736.000000,8.973600e+04,8.973600e+04,8.973600e+04,8.973600e+04,89736.000000,89736.000000,89736.000000,89736.000000,81386.000000,83142.000000,83136.000000,83127.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000,89736.000000
mean,5.953523,5.953322,5.953226,5.953159,5.071152e+05,5.071078e+05,5.070938e+05,5.070821e+05,140.862686,140.849243,140.850327,140.851059,1695.173462,1739.129883,1737.034058,1734.952271,275.085907,275.086090,275.086090,275.085846,38.977650,38.896389,38.725594,38.901039,66.906639,66.996170,67.024910,66.993721,1007.935913,1007.935974,1007.935913,1007.935974,182.450546,180.720367,176.390564,179.161469,8.919071,8.922294,8.921758,8.918437,279.316040,279.315887,279.315491,279.315216,33197.964844,33196.117188,33193.707031,33190.753906,3.031766,3.031960,3.032194,3.032326,56.376804,97.122838,5.997560,292.994650
std,2.688669,2.686662,2.685953,2.686620,8.163501e+05,8.163506e+05,8.163531e+05,8.163515e+05,227.341827,227.347717,227.343414,227.338608,1782.952637,1812.471680,1796.203735,1804.134399,6.813097,6.809192,6.807894,6.809395,60.378723,60.464146,60.671577,60.457279,35.113735,34.500549,34.295116,34.497101,13.185225,13.182809,13.182015,13.182457,102.822189,102.914253,102.787804,102.736809,13.520752,13.518727,13.518906,13.520430,6.507071,6.503327,6.502243,6.503626,18292.039062,17969.498047,17861.337891,17971.714844,1.768140,1.753842,1.748710,1.753654,120.571304,51.539308,3.488807,779.495311
min,0.500000,0.500000,0.500000,0.500000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,27.400000,27.500000,27.500000,27.500000,247.300003,247.500000,247.399994,247.300003,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,941.400024,941.599976,941.500000,941.500000,0.008000,0.002000,0.003000,4.038000,-0.000000,-0.000000,-0.000000,-0.000000,257.899994,258.000000,258.000000,258.000000,130.600006,131.800003,133.000000,134.100006,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000
25%,4.000000,4.000000,4.000000,4.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,577.799988,595.325012,598.375000,595.400024,270.600006,270.600006,270.600006,270.600006,0.000000,0.000000,0.000000,0.000000,41.000000,41.599998,41.599998,41.299999,999.799988,999.799988,999.799988,999.799988,93.066500,91.009003,87.676001,90.112999,-0.00000

In [ ]:
print(x_whole.shape)
print(other_train.shape)

(77044, 48)
(89736, 60)


In [ ]:
print(x_whole.columns.shape)
print(other_train.columns.shape)
x_whole.columns

(48,)
(60,)


Index(['absolute_humidity_2m:gm3', 'air_density_2m:kgm3',
       'clear_sky_energy_1h:J', 'clear_sky_rad:W', 'cloud_base_agl:m',
       'dew_or_rime:idx', 'dew_point_2m:K', 'diffuse_rad:W',
       'diffuse_rad_1h:J', 'direct_rad:W', 'direct_rad_1h:J',
       'effective_cloud_cover:p', 'is_day:idx', 'is_in_shadow:idx',
       'precip_5min:mm', 'precip_type_5min:idx', 'pressure_50m:hPa',
       'prob_rime:p', 'rain_water:kgm2', 'relative_humidity_1000hPa:p',
       'sun_azimuth:d', 'super_cooled_liquid_water:kgm2', 't_1000hPa:K',
       'total_cloud_cover:p', 'visibility:m', 'wind_speed_10m:ms',
       'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms', 'wind_speed_w_1000hPa:ms',
       'location_a', 'location_b', 'location_c', 'sin_day_of_year',
       'cos_day_of_year', 'sin_hour', 'cos_hour', 'sun_product',
       'modified_solar_elevation', 'effective_radiation',
       'time_since_prediction', 'cloud_ratio', 'cloud_cover_over_30%',
       'sun_addition', 'direct_rad_cloud_interaction',
  

# Differences in test data

In [ ]:
X_test_est_combined.describe()

,absolute_humidity_2m:gm3,air_density_2m:kgm3,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,is_day:idx,is_in_shadow:idx,precip_5min:mm,precip_type_5min:idx,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sun_azimuth:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,location_a,location_b,location_c,sin_day_of_year,cos_day_of_year,sin_hour,cos_hour,sun_product,modified_solar_elevation,effective_radiation,time_since_prediction,cloud_ratio,cloud_cover_over_30%,sun_addition,direct_rad_cloud_interaction,modified_solar_elevation_squared,is_freezing,is_snow,is_rain
count,2160.000000,2160.000000,2.160000e+03,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2.160000e+03,2160.000000,2.160000e+03,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2.160000e+03,2.160000e+03,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.0,2160.000000,2160.00000
mean,8.202084,1.232774,1.227747e+06,341.040741,1681.780151,0.036111,280.778992,84.755867,3.051201e+05,114.341972,4.116306e+05,64.090988,0.793519,0.241204,0.007663,0.065972,1008.982666,0.016435,0.013067,70.843506,179.180542,0.064525,284.749359,69.259583,33348.164062,2.945822,1.652465,-0.182477,0.000359,0.333333,0.333333,0.333333,0.431752,-0.855224,-1.562536e-17,-5.509996e-17,14912.679688,0.340791,0.262975,28.494250,0.871397,0.750000,199.097855,7167.174316,0.487801,0.0,0.001852,0.08287
std,2.191513,0.032043,1.101274e+06,306.855469,2062.581299,0.182849,4.364160,78.432159,2.777555e+05,171.784576,6.107051e+05,37.460999,0.385461,0.413672,0.031816,0.235240,9.738119,0.468430,0.051688,15.604269,103.925461,0.109135,5.830846,37.764587,15336.857422,1.714616,2.569777,1.490788,0.005502,0.471514,0.471514,0.471514,0.252718,0.135475,7.072705e-01,7.072705e-01,23051.986328,0.278666,0.285936,6.923792,0.250668,0.433113,214.731079,13825.104492,0.320762,0.0,0.043003,0.27575
min,3.200000,1.142750,0.000000e+00,0.000000,0.000000,-1.000000,268.075012,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,977.575012,0.000000,0.000000,24.599998,13.439500,0.000000,273.700012,0.000000,1532.175049,0.100000,-4.300000,-4.300000,0.000000,0.000000,0.000000,0.000000,-0.006451,-0.999979,-1.000000e+00,-1.000000e+00,0.000000,0.000000,0.000000,16.976944,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000
25%,6.650000,1.209437,4.874178e+04,17.806250,402.631256,0.000000,277.924988,7.975000,3.417926e+04,0.000000,4.922562e+02,30.131249,1.000000,0.000000,0.000000,0.000000,1003.750000,0.000000,0.000000,60.275002,89.979689,0.000000,279.849976,36.625001,19856.712891,1.550000,-0.200000,-1.300000,0.000000,0.000000,0.000000,0.000000,0.232243,-0.972658,-7.071068e-01,-7.071068e-01,0.000000,0.042327,0.001233,22.732500,0.886944,0.750000,8.512500,0.000000,0.205706,0.0,0.000000,0.00000
50%,8.050000,1.238000,9.333753e+05,299.512512,852.362518,0.000000,281.049988,75.224998,2.649308e+05,13.475000,6.548056e+04,77.500000,1.000000,0.000000,0.000000,0.000000,1012.250000,0.000000,0.000000,73.587502,180.959869,0.000000,284.799988,92.237499,37066.824219,2.750000,1.600000,-0.250000,0.000000,0.000000,0.000000,0.000000,0.480303,-0.877060,6.123234e-17,-6.123234e-17,1170.831604,0.338697,0.113782,28.488056,1.000000,1.000000,119.412502,38.066195,0.581975,0.0,0.000000,0.00000
75%,10.000000,1.259500,2.276978e+06,662.018738,2258.043701,0.000000,284.375000,134.418751,4.880685e+05,187.493748,6.558369e+05,99.974998,1.000000,0.250000,0.000000,0.000000,1016.200012,0.000000,0.000000,83

In [ ]:
other_tests = pd.concat([A_X_test_estimated, B_X_test_estimated, C_X_test_estimated])
other_tests.describe()

,absolute_humidity_2m:gm3_0,absolute_humidity_2m:gm3_15,absolute_humidity_2m:gm3_30,absolute_humidity_2m:gm3_45,clear_sky_energy_1h:J_0,clear_sky_energy_1h:J_15,clear_sky_energy_1h:J_30,clear_sky_energy_1h:J_45,clear_sky_rad:W_0,clear_sky_rad:W_15,clear_sky_rad:W_30,clear_sky_rad:W_45,cloud_base_agl:m_0,cloud_base_agl:m_15,cloud_base_agl:m_30,cloud_base_agl:m_45,dew_point_2m:K_0,dew_point_2m:K_15,dew_point_2m:K_30,dew_point_2m:K_45,diffuse_rad:W_0,diffuse_rad:W_15,diffuse_rad:W_30,diffuse_rad:W_45,effective_cloud_cover:p_0,effective_cloud_cover:p_15,effective_cloud_cover:p_30,effective_cloud_cover:p_45,sfc_pressure:hPa_0,sfc_pressure:hPa_15,sfc_pressure:hPa_30,sfc_pressure:hPa_45,sun_azimuth:d_0,sun_azimuth:d_15,sun_azimuth:d_30,sun_azimuth:d_45,sun_elevation_is_day_0,sun_elevation_is_day_15,sun_elevation_is_day_30,sun_elevation_is_day_45,t_1000hPa:K_0,t_1000hPa:K_15,t_1000hPa:K_30,t_1000hPa:K_45,visibility:m_0,visibility:m_15,visibility:m_30,visibility:m_45,wind_speed_10m:ms_0,wind_speed_10m:ms_15,wind_speed_10m:ms_30,wind_speed_10m:ms_45,direct_rad:W_max,days_from_apex,hours_from_apex,id,prediction
count,2160.000000,2160.000000,2160.000000,2160.000000,2.160000e+03,2.160000e+03,2.160000e+03,2.160000e+03,2160.000000,2160.000000,2160.000000,2160.000000,1879.000000,1937.000000,1937.000000,1937.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.0
mean,8.206482,8.203751,8.200879,8.197222,1.227747e+06,1.227748e+06,1.227744e+06,1.227751e+06,341.056641,341.017426,341.027771,341.061157,1797.160156,1882.277466,1868.517578,1854.756348,280.783203,280.781799,280.777832,280.773163,84.915688,84.771759,84.511574,84.824440,64.113792,64.143097,64.105553,64.001480,1015.070374,1015.052795,1015.034851,1015.019653,183.166199,178.425781,175.685333,179.444855,20.981165,20.999121,20.982639,20.991322,284.737732,284.746521,284.751953,284.761292,33304.636719,33332.203125,33361.906250,33393.902344,2.946759,2.946250,2.945278,2.945000,127.607353,18.566667,6.000000,1079.500000,0.0
std,2.201396,2.194498,2.189956,2.188581,1.104469e+06,1.104466e+06,1.104470e+06,1.104464e+06,307.729095,307.772369,307.762604,307.724518,2046.394409,2119.177002,2074.760498,2062.211670,4.378817,4.369216,4.361317,4.359707,78.422508,78.761269,79.599770,78.715187,37.947498,37.571831,37.476212,37.674469,9.840412,9.839953,9.837538,9.836187,109.193207,109.361115,109.129372,109.135262,17.795698,17.775604,17.793594,17.783812,5.839595,5.833704,5.829640,5.833142,15624.633789,15395.229492,15317.976562,15397.651367,1.733865,1.719152,1.712798,1.721526,181.000122,12.473342,3.488883,623.682612,0.0
min,3.200000,3.200000,3.200000,3.200000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,29.799999,30.200001,30.200001,30.200001,268.000000,268.100006,268.100006,268.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,983.500000,983.400024,983.400024,983.299988,8.270000,0.048000,2.250000,5.716000,-0.000000,-0.000000,-0.000000,-0.000000,273.700012,273.700012,273.700012,273.700012,874.400024,1424.800049,1400.400024,1137.300049,0.000000,0.100000,0.100000,0.100000,0.000000,1.000000,0.000000,0.000000,0.0
25%,6.600000,6.600000,6.600000,6.700000,6.433812e+04,4.838737e+04,5.208360e+04,6.312340e+04,13.650000,16.400000,16.750000,12.700000,486.899994,512.000000,506.899994,508.100006,277.899994,277.899994,277.899994,278.000000,6.925000,6.575000,8.325000,7.100000,30.700001,30.500000,31.100000,29.799999,1009.799988,1009.799988,1009.700012,1009.700012,85.359253,78.783001,78.778500,82.055502,1.368750,2.059000,1.455000,1.3

In [ ]:
# Find amount of missing values for X_train_estimated_a, X_train_estimated_b, X_train_estimated_c, X_train_observed_a, X_train_observed_b, X_train_observed_c, X_test_estimated_a, X_test_estimated_b, X_test_estimated_c
print("Estimated")
print(X_train_estimated_a.isnull().sum())
print(X_train_estimated_b.isnull().sum())
print(X_train_estimated_c.isnull().sum())
print("Observed")
print(X_train_observed_a.isnull().sum())
print(X_train_observed_b.isnull().sum())
print(X_train_observed_c.isnull().sum())
print("Test")
print(X_test_estimated_a.isnull().sum())
print(X_test_estimated_b.isnull().sum())
print(X_test_estimated_c.isnull().sum())


Estimated
date_calc                             0
date_forecast                         0
absolute_humidity_2m:gm3              0
air_density_2m:kgm3                   0
ceiling_height_agl:m               3919
clear_sky_energy_1h:J                 0
clear_sky_rad:W                       0
cloud_base_agl:m                   2094
dew_or_rime:idx                       0
dew_point_2m:K                        0
diffuse_rad:W                         0
diffuse_rad_1h:J                      0
direct_rad:W                          0
direct_rad_1h:J                       0
effective_cloud_cover:p               0
elevation:m                           0
fresh_snow_12h:cm                     0
fresh_snow_1h:cm                      0
fresh_snow_24h:cm                     0
fresh_snow_3h:cm                      0
fresh_snow_6h:cm                      0
is_day:idx                            0
is_in_shadow:idx                      0
msl_pressure:hPa                      0
precip_5min:mm                